# Прогноз инфляции. Кейс 1 для хакатона. Сентябрь 2022

Команда: Цикличность действий

**В данном ноутбуке приведен конечный результат кода, однако было несколько итераций до его получения:**

  1. В первой итерации программы была использована модель ARIMA, которая делала прогноз только исходя из целевой функции. Подход был не верный и данное решение было принято отбросить.
  2. Во второй итерации программы упор был сконцентрирован на EDA - анализе входных данных из датасета DS_test. Изучалась предметная область инфляции и как признаки из набора данных могут на нее влиять. Набор данных был уменьшен, в качестве основного признака было принято решение добавить признак PCI_Multi - коэффицент\ множитель индекса. Перед тем как посчитать множитель индекса, ряд был приведен к регулярному (убраны временные пробелы, на месте пустых значений цены - значения предыдущих показателей (полагаем, что цена не менялась за промежуток времени, когда не было показаний)). И были убраны дубликаты дат с сохранением последнего значения цены, подход может быть слегка грубый, но значительную потерю в точности не дал. После, данные были сгруппированы в строки по месяцам (+ годам), при этом множитель индекса взялся как среднее его значение за месяц (mean value). Для обучения было выбрано использовать RandomForestRegressor из-за популярности использования деревьев и их точности. Результат получился удовлетворительным (MAE = 0.3), но хотелось улучшить модель.
  3. В третьей итерации было принято решение использовать для обучения blending ensemble, EDA остался тем же. В качестве моделей для бленда были выбраны RandomForestRegressor, XGBRegressor, LinearRegression, LASSO. Результат такого бленда получился лучше - MAE = 0.2. 
  4. В четвертой итерации было решено взять модели из бленда и проверить их по отдельности. И лучшим решением оказалось обучение модели XGBRegressor с метрикой MAE = 0.03. Было принято оставить эту модель и заняться тюном параметров. Однако, тьюн ухудшил показания метрики (возможно ошибка программиста с тьюном , но результат таков) и было решено оставить XGBRegressor с параметрами (objective ='reg:squarederror', n_estimators = 10, seed = 123), как было использовано изначально. 👍


---


  **Также в ноутбуке есть 2 скрипта - forecast.py и test_forecast.py:**

  **forecast.py** - на вход аргументами подается путь к файлу DS_train, слово 'learn' и путь к файлу с целевыми переменными. Скрипт повторяет действия ноутбука - обучает модель, сохраняет ее в json и сохраняет датафрейм с периодом + целевыми переменными (+ ЦП, которая была предсказана) также в формат json. Этот скрипт используется для сайта - интерфеса, который показывает работу разработаного кода. *Cлово 'forecast' в аргументах - наработка для дальнейшего развития, планировалась сделать несколько вариантов : learn - для обучения модели с нуля, add - для дополнения модели новыми данными и дообучения модели, predict - для загрузки части данных текущего месяца и получения прогноза по ним*

  **test_forecast.py** - скрипт, написаный для прохождения периода тестирования на новых данных и адаптации, единственная разница - вместо mae используется предложенная метрика score и возвращается файл в формате .txt с метрикой. Как аргументы  для вызова используются путь к файлу с параметрами, название файла для сохранения скора, путь к файлу с целевыми функциями





### Маунт для Google Colab

In [39]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Загрузка данных

In [40]:
# Импорт библиотек
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error # метрика
import xgboost as xg  # модель


In [41]:
# загружаем данные, меняем отображение временной метки на год-месяц
ds=pd.read_csv("/content/drive/MyDrive/ИИ_прогнозирует_инфляцию/DS_train(2020-06--2022-06-01).csv", sep='\t', index_col='DateObserve')
ds.index = pd.to_datetime(ds.index).to_period('M')
ds

,WebPriceId,StockStatus,CurrentPrice
DateObserve,,,
2020-06,1,InStock,49.0
2020-09,1,OutOfStock,NaN
2020-09,1,InStock,49.0
2020-09,1,OutOfStock,NaN
2020-09,1,InStock,49.0
...,...,...,...
2022-04,10001000,InStock,291.0
2022-05,10001000,InStock,242.0
2022-05,10001000,InStock,231.0


### EDA, отбор строк, удаление пропусков, заполнение временных меток до регулярности, подсчет множителя ИПЦ

In [42]:
ds.drop_duplicates(subset=['WebPriceId','CurrentPrice'],keep= False, inplace=True)  # убираем строки с одинаковой ценой
ds = ds[['WebPriceId','CurrentPrice']]  # убираем столбец СтокСтатус
ds = ds.dropna()  # убираем строки с NAN
ds = ds.reset_index() # сбрасываем индекс для дальнейшего анализа
ds = ds.drop_duplicates(subset=['DateObserve','WebPriceId'], keep='last') # убираем строки с одинаковыми id по дате, оставляем последние вхождения
ds=ds.set_index(['DateObserve','WebPriceId']).unstack(fill_value=0).asfreq('M', fill_value=0).stack().sort_index(level=1).reset_index() # заполняем ряды до регулярности
ds.CurrentPrice = ds.CurrentPrice.replace(to_replace=0, method ='ffill')  # заполняем нули предыдущими значениями
ds = ds[(ds != 0).all(1)] # выкидываем значения с 0 - это первые временные значения, которые не повлияют на прогноз
ds

,DateObserve,WebPriceId,CurrentPrice
4,2020-10,2,48.0
5,2020-11,2,48.0
6,2020-12,2,49.0
7,2021-01,2,51.0
8,2021-02,2,51.0
...,...,...,...
29715475,2022-01,1250999,2484.0
29715476,2022-02,1250999,2484.0
29715477,2022-03,1250999,2484.0
29715478,2022-04,1250999,2484.0


In [43]:
#Расчет множителя ИПЦ
ds['CPI_multi'] = ds['CurrentPrice'].iloc[-1] / ds['CurrentPrice']
df = ds.groupby('DateObserve')['CPI_multi'].mean()
df = df.to_frame(name='cpi_multi')
df

,cpi_multi
DateObserve,
2020-06,9.332595
2020-07,28.423599
2020-08,22.090222
2020-09,18.855109
2020-10,17.106960
2020-11,14.347905
2020-12,13.635527
2021-01,13.077064
2021-02,12.623905


### Загрузка целевого показателя, объединение наборов данных

In [44]:
data = pd.read_excel("/content/drive/MyDrive/ИИ_прогнозирует_инфляцию/Y_train.xlsx", index_col='ИПЦ, мом')
data = data.T
data = data.set_index('Период')
data = data[['Целевой показатель']]
data.index = pd.to_datetime(data.index).to_period('M')
data

"ИПЦ, мом",Целевой показатель
Период,
2020-06,0.28
2020-07,-0.065
2020-08,-0.005
2020-09,0.315
2020-10,0
2020-11,0.375
2020-12,0.35
2021-01,0.38
2021-02,0.685


In [45]:
ds = df.join(data, how='outer')
ds

,cpi_multi,Целевой показатель
DateObserve,,
2020-06,9.332595,0.28
2020-07,28.423599,-0.065
2020-08,22.090222,-0.005
2020-09,18.855109,0.315
2020-10,17.106960,0
2020-11,14.347905,0.375
2020-12,13.635527,0.35
2021-01,13.077064,0.38
2021-02,12.623905,0.685


### Обучение

In [46]:
# разделяем набор данных на тестовую и тренировочную выборки, в тестовой 1 месяц
train, test = ds[0:(len(ds)-1)], ds[(len(ds)-1):]

In [47]:
print('Тренировочная выборка: ', train.shape)
print('Тестовая выборка: ', test.shape)

Тренировочная выборка:  (23, 2)
Тестовая выборка:  (1, 2)


In [48]:
# Разделяем наборы данных на таргет и параметры
y_train= train['Целевой показатель']
x_train = train.drop(labels = ['Целевой показатель'], axis=1)

y_test = test['Целевой показатель']
x_test = test.drop(labels = ['Целевой показатель'], axis=1)

In [49]:
# используем XGBRegressor для обучения
model = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 10, seed = 123)
# фит признаков
model.fit(x_train, y_train)
# делаем предсказание на 1 значение
pred = model.predict(x_test)

In [50]:
# смотрим результат по метрике MAE
mean_absolute_error(test['Целевой показатель'], pred)

0.03187578678131109

MAE = 0.0319 - модель сделала хорошее предсказание

In [ ]:
# сохраняем обученную модель
model.save_model("model.json")

In [51]:
# сохраняем результат предсказания вместе с имеющимися значениями ЦФ в датафрейм
ds = ds.reset_index()

y = y_train.reset_index()
y = np.array(y[['Целевой показатель']])
y = np.append(y,pred[0])

tdf = pd.DataFrame({'период':np.array(ds.DateObserve), 'целевая функция':y})
tdf

,период,целевая функция
0,2020-06,0.28
1,2020-07,-0.065
2,2020-08,-0.005
3,2020-09,0.315
4,2020-10,0
5,2020-11,0.375
6,2020-12,0.35
7,2021-01,0.38
8,2021-02,0.685
9,2021-03,0.255


In [ ]:
# сохраняем датафрейм как файл json
tdf.to_json(f'result.json',orient = 'split',index = False)

### Скрипт для получения результатов прогноза в формате json и обученной модели в формате json

In [36]:
%%writefile forecast.py
# Пример запуска: run test_forecast.py "DS_train(2020-06--2022-06-01).csv" "learn" "Y_train.xlsx"
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import argparse

# импорт моделей
import xgboost as xg

# импорт метрики
from sklearn.metrics import mean_absolute_error

def run_neural_network(path: str, is_forecast: str, target_path: str):
    # загружаем данные, устанавливаем месячный период
    ds=pd.read_csv(path, sep='\t', index_col='DateObserve')
    ds.index = pd.to_datetime(ds.index).to_period('M')

    # Убираем из данных пропуски, дубликаты, заполняем до однородных временных меток
    ds.drop_duplicates(subset=['WebPriceId','CurrentPrice'],keep= False, inplace=True)
    ds = ds[['WebPriceId','CurrentPrice']]
    ds = ds.dropna()
    ds = ds.reset_index()
    ds = ds.drop_duplicates(subset=['DateObserve','WebPriceId'], keep='last')
    ds=ds.set_index(['DateObserve','WebPriceId']).unstack(fill_value=0).asfreq('M', fill_value=0).stack().sort_index(level=1).reset_index()
    ds.CurrentPrice = ds.CurrentPrice.replace(to_replace=0, method ='ffill')
    ds = ds[(ds != 0).all(1)]

    #Расчет множителя ИПЦ
    ds['CPI_multi'] = ds['CurrentPrice'].iloc[-1] / ds['CurrentPrice']
    df = ds.groupby('DateObserve')['CPI_multi'].mean()

    df = df.to_frame(name='cpi_multi')

    if (is_forecast =="learn"):
      # загружаем таргет 

      data = pd.read_excel(target_path, index_col='ИПЦ, мом')
      data = data.T
      data = data.set_index('Период')
      data = data[['Целевой показатель']]
      data.index = pd.to_datetime(data.index).to_period('M')
      # соединяем таблицы
      ds = df.join(data, how='outer')

      """ разделяем на тестовую и тренировочную выборки"""

      train, test = ds[0:(len(data)-1)], ds[(len(data)-1):]

      # Делим выборки на таргет и фичи
      y_train= train['Целевой показатель']
      x_train = train.drop(labels = ['Целевой показатель'], axis=1)

      y_test = test['Целевой показатель']
      x_test = test.drop(labels = ['Целевой показатель'], axis=1)


      model = xg.XGBRegressor(objective ='reg:squarederror',
                      n_estimators = 10, seed = 123)

      # Фит
      model.fit(x_train, y_train)
      
      # Предсказание
      pred = model.predict(x_test)
      #mean_absolute_error(test['Целевой показатель'], pred)
      model.save_model("model.json")
 
    # сохраняем и возвращаем результат в виде таблицы

      ds = ds.reset_index()
      y = y_train.reset_index()
      y = np.array(y[['Целевой показатель']])
      y = np.append(y,pred[0])

      tdf = pd.DataFrame({'период':np.array(ds.DateObserve), 'целевая функция':y})

      tdf.to_json(f'result.json',orient = 'split',index = False)
      #return tdf.to_json(f'result.json',orient = 'split',index = False)


if __name__ == "__main__":
    """получение пути к файлу и имени датасета"""
    parser = argparse.ArgumentParser(description='Передайте путь к датасету , слово learn и путь к целевым функциям')
    parser.add_argument('path', type=str)
    parser.add_argument('forecast', type=str)
    parser.add_argument('target_path', type=str)
    args = parser.parse_args()

    path = args.path
    is_forecast = args.forecast # = "learn"
    target_path = args.target_path

    run_neural_network(path, is_forecast, target_path)

Overwriting forecast.py


In [ ]:
%run forecast.py "/content/drive/MyDrive/ИИ_прогнозирует_инфляцию/DS_train(2020-06--2022-06-01).csv" "learn" "/content/drive/MyDrive/ИИ_прогнозирует_инфляцию/Y_train.xlsx"

### Скрипт для прохождения периода проверки адаптивности и тестирования модели

Файл корректировался в зависимости от Y_train т.к. Y_train_adapt имеет другое название колонки с целевой переменной

In [38]:
%%writefile test_forecast.py
# Пример запуска: run test_forecast.py "DS_train(2020-06--2022-06-01).csv" "test_1.txt" "Y_train.xlsx"
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import argparse

# импорт моделей
import xgboost as xg

def run_model(path: str, filename:str, target_path: str):
    # загружаем данные, устанавливаем месячный период
    ds=pd.read_csv(path, sep='\t', index_col='DateObserve')
    ds.index = pd.to_datetime(ds.index).to_period('M')

    # Убираем из данных пропуски, дубликаты, заполняем до однородных временных меток
    ds.drop_duplicates(subset=['WebPriceId','CurrentPrice'],keep= False, inplace=True)
    ds = ds[['WebPriceId','CurrentPrice']]
    ds = ds.dropna()
    ds = ds.reset_index()
    ds = ds.drop_duplicates(subset=['DateObserve','WebPriceId'], keep='last')
    ds=ds.set_index(['DateObserve','WebPriceId']).unstack(fill_value=0).asfreq('M', fill_value=0).stack().sort_index(level=1).reset_index()
    ds.CurrentPrice = ds.CurrentPrice.replace(to_replace=0, method ='ffill')
    ds = ds[(ds != 0).all(1)]

    #Расчет множителя ИПЦ
    ds['CPI_multi'] = ds['CurrentPrice'].iloc[-1] / ds['CurrentPrice']
    df = ds.groupby('DateObserve')['CPI_multi'].mean()

    df = df.to_frame(name='cpi_multi')

    # загружаем таргет 

    data = pd.read_excel(target_path, index_col='ИПЦ, мом')
    data = data.T
    data = data.set_index('Период')
    #if 'adp' in filename:
    data = data[['Целевой показатель (для проверки адаптивности)']]
    #else:
    #  data = data[['Целевой показатель']]
    data.index = pd.to_datetime(data.index).to_period('M')
    # соединяем таблицы
    ds = df.join(data, how='outer')

    """ разделяем на тестовую и тренировочную выборки"""

    train, test = ds[0:(len(data)-1)], ds[(len(data)-1):]

    # Убираем из тренировочной выборки таргет
    y_train= train['Целевой показатель (для проверки адаптивности)']
    x_train = train.drop(labels = ['Целевой показатель (для проверки адаптивности)'], axis=1)

    y_test = test['Целевой показатель (для проверки адаптивности)']
    x_test = test.drop(labels = ['Целевой показатель (для проверки адаптивности)'], axis=1)


    model = xg.XGBRegressor(objective ='reg:squarederror',
                      n_estimators = 10, seed = 123)

    # Fitting the model
    model.fit(x_train, y_train)
      
    # Predict the model
    pred = model.predict(x_test)

    model.save_model("test_model.json")
 
    # записываем результат в файл

    f = open(filename,"w+")
    f.write(str(pred[0]))
    f.close()

if __name__ == "__main__":
    """получение пути к файлу и имени датасета"""
    parser = argparse.ArgumentParser(description='Передайте путь к датасету и целевой функции, название файла для сохранения')
    parser.add_argument('path', type=str)
    parser.add_argument('filename', type=str)
    parser.add_argument('target_path', type=str)
    args = parser.parse_args()

    path = args.path
    filename = args.filename
    target_path = args.target_path

    run_model(path,filename, target_path)

Overwriting test_forecast.py


In [ ]:
%run test_forecast.py "/content/drive/MyDrive/ИИ_прогнозирует_инфляцию/DS_train(2020-06--2022-06-01).csv" "test_1.txt" "/content/drive/MyDrive/ИИ_прогнозирует_инфляцию/Y_train.xlsx"

In [35]:
%run test_forecast.py "/content/drive/MyDrive/ИИ_прогнозирует_инфляцию/DS_test2(2020-06--2022-06-21).csv" "test_adp_2.txt" "/content/drive/MyDrive/ИИ_прогнозирует_инфляцию/Y_train_adaptive.xlsx"